In [56]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected

In [57]:
userActionHistory = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/userActionHistory.csv',
                      header=0,encoding='gbk' )
predUser_item = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/predDF.csv',
                      header=0,encoding='gbk' )

In [44]:
#index

In [58]:
user_count=dict()
user2idx=dict()
idx2user=[]
for user_id in userActionHistory['user_id']:
    user_count.setdefault(user_id,0)
    user_count[user_id]+=1
idx=0   
for user_id in user_count.keys():
    user2idx[user_id]=idx
    idx+=1
    idx2user.append(user_id)
userDF = np.array([user2idx[user_id] for user_id in userActionHistory['user_id']],dtype=np.int32)
userTrain = userDF[:1700000]
userTest = userDF[1700000:]

predUserDF = np.array([user2idx[user_id] for user_id in predUser_item['user_id']],dtype=np.int32)

In [59]:
sku_count=dict()
sku2idx=dict()
idx2sku=[]
for sku_id in userActionHistory['sku_id']:
    sku_count.setdefault(sku_id,0)
    sku_count[sku_id]+=1
idx=0   
for sku_id in sku_count.keys():
    sku2idx[sku_id]=idx
    idx+=1
    idx2sku.append(sku_id)
skuDF = np.array([sku2idx[sku_id] for sku_id in userActionHistory['sku_id']],dtype=np.int32)
skuTrain = skuDF[:1700000]
skuTest = skuDF[1700000:]
predskuDF = np.array([sku2idx[sku_id] for sku_id in predUser_item['buyedSku']],dtype=np.int32)


KeyError: 'sku_id'

In [68]:
targDF = np.array([types.split("#") for types in userActionHistory['V1']],dtype=np.float32)
targTrain = targDF[:1700000]
targTest= targDF[1700000:]

In [69]:
##construct graph
Prod_size = len(idx2sku)
prod_embedding_size = 100
user_size = len(idx2user)
user_embedding_size =100
n_output=6
alpha=0.001

In [70]:
prod_embeddings  = tf.Variable(tf.random_uniform([Prod_size,prod_embedding_size], -1.0, 1.0))
user_embeddings  = tf.Variable(tf.random_uniform([user_size,user_embedding_size], -1.0, 1.0))

In [71]:
prod_idx  = tf.placeholder(tf.int32, shape=[None])
user_idx  = tf.placeholder(tf.int32, shape=[None])
 
prod_embed  = tf.nn.embedding_lookup(prod_embeddings , prod_idx)
user_embed  = tf.nn.embedding_lookup(user_embeddings , user_idx)
 
y = tf.placeholder(tf.float32,[None,n_output])

In [72]:
final_feat=tf.concat([prod_embed,user_embed], 1) 

In [73]:
logits = fully_connected(final_feat, n_output, activation_fn=None)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=alpha)
training_op = optimizer.minimize(loss)
actLogit = tf.sigmoid(logits)
prediction = tf.round(actLogit)
predictions_correct = tf.cast(tf.equal(prediction, y),tf.float32)
accuracy = tf.reduce_mean(predictions_correct)
init = tf.global_variables_initializer()

In [74]:
saver = tf.train.Saver()

In [75]:
n_epochs=1
batch_size=1000
batch_num = len(targTrain)//batch_size+1
 
load_path = "old"
save_path = "old"

In [77]:
with tf.Session() as sess:    
    init.run()
    for epoch in range(n_epochs):
        for batch in range(batch_num):
            prod_batch       = skuTrain[batch*batch_size:(batch+1)*batch_size]
            user_batch       = userTrain[batch*batch_size:(batch+1)*batch_size]   
            y_batch          = targTrain[batch*batch_size:(batch+1)*batch_size]
            sess.run(training_op, feed_dict={
                prod_idx: prod_batch,user_idx:user_batch,y:y_batch})
            if batch%100==0:
                train_acc = accuracy.eval(feed_dict={ prod_idx: prod_batch,user_idx:user_batch,y:y_batch}) 
                Test_acc = accuracy.eval(feed_dict={ prod_idx: skuTest,user_idx:userTest,y:targTest}) 
                print(train_acc,Test_acc)
    save_path = saver.save(sess,"C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/tfsave/test.ckpt")


0.5015 0.499289
0.866333 0.868878
0.9405 0.942634
0.9495 0.947176
0.938167 0.947735
0.947333 0.947851
0.943833 0.947886
0.943667 0.947907
0.948833 0.947914
0.947833 0.947974
0.944833 0.948199
0.940333 0.948544
0.950667 0.949012
0.947 0.949497
0.947833 0.949958
0.951167 0.950296
0.9515 0.950665


In [92]:
chunk_size=100000
chunk_num = len(predskuDF)//chunk_size+1
with tf.Session() as sess:
    saver.restore(sess,"C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/tfsave/test.ckpt")
    for chunk in range(chunk_num):
        if chunk%100==0:
            print(chunk)
        tmp = actLogit.eval(feed_dict={ prod_idx: predskuDF[chunk*chunk_size:(chunk+1)*chunk_size]
                                           ,user_idx: predUserDF[chunk*chunk_size:(chunk+1)*chunk_size]})
        if chunk==0:
            predict = tmp
        else:
            predict = np.concatenate((predict,tmp),axis=0)

0
100
200
300
400
500
600
700


In [106]:
out = pd.DataFrame(predict)

In [127]:
p = np.where(out[3]>0.6)[0]

In [131]:
user_id = np.array(predUser_item["user_id"])

In [132]:
sku_id = np.array(predUser_item["buyedSku"])

In [137]:
sku_id =sku_id[p]

In [138]:
user_id = user_id[p]

In [144]:
out = pd.DataFrame(user_id,sku_id)

In [147]:
out=out.drop_duplicates()

In [148]:
 out.to_csv("D:/out0331.csv")